In [1]:
import xgboost as xgb
import lightgbm as lgb
import catboost as cat

In [2]:
import pandas as pd
import joblib

In [3]:
import numpy as np
import random

In [4]:
np.random.seed(42)
random.seed(42)

In [5]:
def read_merge_data(data_type="training"):
    #
    Dfs = []
    train_df = []
    train_output = pd.read_csv("./data_phase_one/training_output.csv", header=None)
    coords = pd.read_csv("data_phase_one/coords.csv", header=None)
    for name in ["mslp", "sst", "ssta", "t2m"]:
        df = pd.read_csv(f"./data_phase_one/{data_type}_input_{name}.csv", header=None)
        print("Shape df", df.shape)
        Dfs.append(df)
    j = 0
    for i in range(0, df.shape[0]-12, 1):
        feat = pd.concat([coords, Dfs[0].iloc[i:i+12], Dfs[1].iloc[i:i+12], Dfs[2].iloc[i:i+12], Dfs[3].iloc[i:i+12]], axis=0).reset_index(drop=True)
        tgt = train_output.iloc[i+11]
        tgt = tgt.T.reset_index(drop=True)
        feat = feat.T.reset_index(drop=True)
        feat = pd.concat([feat, tgt], axis=1)
        assert feat.shape[1]==51
        feat.columns = [f"feat_{k}" for k in range(feat.shape[1]-1)] + ["target"]
        if j < 3:
            print(feat.columns)
        train_df.append(feat)
        j += 1
    train_df = pd.concat(train_df, axis=0)
    #Dfs.index = [f"col_{i}" for i in range(848)]
    return train_df

In [6]:
data = read_merge_data()

Shape df (849, 5774)
Shape df (849, 5774)
Shape df (849, 5774)
Shape df (849, 5774)
Index(['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6',
       'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12',
       'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18',
       'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24',
       'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'feat_30',
       'feat_31', 'feat_32', 'feat_33', 'feat_34', 'feat_35', 'feat_36',
       'feat_37', 'feat_38', 'feat_39', 'feat_40', 'feat_41', 'feat_42',
       'feat_43', 'feat_44', 'feat_45', 'feat_46', 'feat_47', 'feat_48',
       'feat_49', 'target'],
      dtype='object')
Index(['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6',
       'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12',
       'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18',
       'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23'

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

In [9]:
def normalize_data(X_train, X_val):
    Scaler = StandardScaler()
    data1 = Scaler.fit_transform(X_train)
    data2 = Scaler.fit_transform(X_val)
    return data1, data2

In [10]:
cv = KFold(n_splits=5, shuffle=True)

In [11]:
params_catb = dict(
task_type="CPU",
devices='1',
reg_lambda=0.438, learning_rate=0.1,
max_depth=8, min_data_in_leaf=50,
n_estimators=2000, verbose=50,
objective='RMSE')

In [12]:
params_lgb = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": "rmse",
    "max_depth": 8,
    'num_leaves': 255,
    "learning_rate": 0.01,
    "n_estimators": 2000,
    #"colsample_bytree": 0.8, 
    #"colsample_bynode": 0.8,
    "verbose": -1,
    "device": "cpu"}
#,"gpu_device_id": 1}

In [13]:
for state, (train_idx, val_idx) in enumerate(cv.split(data)):
    X_train = data.drop(columns=["target"]).iloc[train_idx].values
    X_val = data.drop(columns=["target"]).iloc[val_idx].values
    y_train = data["target"].iloc[train_idx].values
    y_val = data["target"].iloc[val_idx].values
    print(f"\nTrain: {len(y_train)}, Val: {len(y_val)}\n")
    X_train, X_val = normalize_data(X_train, X_val)
    lgb_model = lgb.LGBMRegressor(**params_lgb, random_state=state)
    lgb_model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)])
    joblib.dump(lgb_model, f"./kfold_trained/lgb_model_fold{state}.joblib")


Train: 3866270, Val: 966568


Train: 3866270, Val: 966568


Train: 3866270, Val: 966568


Train: 3866271, Val: 966567


Train: 3866271, Val: 966567



In [14]:
for state, (train_idx, val_idx) in enumerate(cv.split(data)):
    X_train = data.drop(columns=["target"]).iloc[train_idx].values
    X_val = data.drop(columns=["target"]).iloc[val_idx].values
    y_train = data["target"].iloc[train_idx].values
    y_val = data["target"].iloc[val_idx].values
    print(f"\nTrain: {len(y_train)}, Val: {len(y_val)}\n")
    X_train, X_val = normalize_data(X_train, X_val)
    catb_model = cat.CatBoostRegressor(**params_catb, random_state=state)
    catb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)])
    joblib.dump(catb_model, f"./kfold_trained/catb_model_fold{state}.joblib")


Train: 3866270, Val: 966568

0:	learn: 0.6049920	test: 0.6063060	best: 0.6063060 (0)	total: 144ms	remaining: 4m 47s
50:	learn: 0.5302618	test: 0.5325222	best: 0.5325222 (50)	total: 4.58s	remaining: 2m 54s
100:	learn: 0.5256719	test: 0.5283223	best: 0.5283223 (100)	total: 8.53s	remaining: 2m 40s
150:	learn: 0.5227794	test: 0.5257284	best: 0.5257284 (150)	total: 12.7s	remaining: 2m 35s
200:	learn: 0.5203494	test: 0.5237294	best: 0.5237294 (200)	total: 16.9s	remaining: 2m 30s
250:	learn: 0.5183086	test: 0.5220537	best: 0.5220537 (250)	total: 20.9s	remaining: 2m 25s
300:	learn: 0.5164816	test: 0.5205960	best: 0.5205960 (300)	total: 24.8s	remaining: 2m 19s
350:	learn: 0.5148668	test: 0.5192960	best: 0.5192960 (350)	total: 28.7s	remaining: 2m 14s
400:	learn: 0.5133017	test: 0.5180607	best: 0.5180607 (400)	total: 32.7s	remaining: 2m 10s
450:	learn: 0.5118236	test: 0.5169268	best: 0.5169268 (450)	total: 36.7s	remaining: 2m 6s
500:	learn: 0.5105610	test: 0.5159400	best: 0.5159400 (500)	total: 

300:	learn: 0.5168669	test: 0.5193946	best: 0.5193946 (300)	total: 48.2s	remaining: 4m 31s
350:	learn: 0.5152529	test: 0.5181238	best: 0.5181238 (350)	total: 54.7s	remaining: 4m 16s
400:	learn: 0.5136603	test: 0.5168866	best: 0.5168866 (400)	total: 58.9s	remaining: 3m 55s
450:	learn: 0.5122559	test: 0.5158119	best: 0.5158119 (450)	total: 1m 3s	remaining: 3m 37s
500:	learn: 0.5109085	test: 0.5148227	best: 0.5148227 (500)	total: 1m 7s	remaining: 3m 21s
550:	learn: 0.5096174	test: 0.5138763	best: 0.5138763 (550)	total: 1m 11s	remaining: 3m 8s
600:	learn: 0.5083085	test: 0.5128749	best: 0.5128749 (600)	total: 1m 15s	remaining: 2m 56s
650:	learn: 0.5070917	test: 0.5119814	best: 0.5119814 (650)	total: 1m 20s	remaining: 2m 46s
700:	learn: 0.5059266	test: 0.5111649	best: 0.5111649 (700)	total: 1m 24s	remaining: 2m 36s
750:	learn: 0.5048350	test: 0.5103571	best: 0.5103571 (750)	total: 1m 28s	remaining: 2m 27s
800:	learn: 0.5037622	test: 0.5096148	best: 0.5096148 (800)	total: 1m 33s	remaining: 2

600:	learn: 0.5084365	test: 0.5107997	best: 0.5107997 (600)	total: 49.6s	remaining: 1m 55s
650:	learn: 0.5072799	test: 0.5099878	best: 0.5099878 (650)	total: 53.8s	remaining: 1m 51s
700:	learn: 0.5060977	test: 0.5091828	best: 0.5091828 (700)	total: 58.1s	remaining: 1m 47s
750:	learn: 0.5049543	test: 0.5083626	best: 0.5083626 (750)	total: 1m 2s	remaining: 1m 43s
800:	learn: 0.5038366	test: 0.5075666	best: 0.5075666 (800)	total: 1m 6s	remaining: 1m 39s
850:	learn: 0.5027847	test: 0.5068335	best: 0.5068335 (850)	total: 1m 10s	remaining: 1m 35s
900:	learn: 0.5017352	test: 0.5061017	best: 0.5061017 (900)	total: 1m 14s	remaining: 1m 31s
950:	learn: 0.5007787	test: 0.5054286	best: 0.5054286 (950)	total: 1m 18s	remaining: 1m 26s
1000:	learn: 0.4998273	test: 0.5047839	best: 0.5047839 (1000)	total: 1m 22s	remaining: 1m 22s
1050:	learn: 0.4987845	test: 0.5040606	best: 0.5040606 (1050)	total: 1m 27s	remaining: 1m 18s
1100:	learn: 0.4978442	test: 0.5034166	best: 0.5034166 (1100)	total: 1m 31s	remai

In [ ]:
n = 1000

In [ ]:

pred_catb = catb_model.predict(X_val[:n])
pred_lgb = lgb_model.predict(X_val[:n])

In [ ]:

print("catboost error:", np.linalg.norm(y_val[:n]-pred_catb))
print("lgbm error:", np.linalg.norm(y_val[:n]-pred_lgb))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(y_val[:n])
plt.plot(pred_lgb)
plt.plot(pred_catb)
plt.legend(["true", "pred_lgb", "pred_catb"])
plt.show()